# Global variable

Ce n'est pas idéal, mais c'est une solution temporaire pour choisir tout les paramètres au début de l'analyse sans avoir à les chercher dans les différentes cellules...

In [ ]:
directory = "/media/admin-suz/Datas/Comparative-analysis_FoldsLeg/fold-10/1160/new"


SKELETON_NAME     = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_006-GB1.fits_c400.up.NDskl.BRK.S006.a.NDskl"
IMAGE_NAME        = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_006.tif"
IMAGE_NAME_MYOSIN = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_006.tif"

# Pixel/Voxel size
# Airyscan
X_SIZE = 0.1832311/2
Y_SIZE = 0.1832311/2
Z_SIZE = 0.2201818

# Confocal
# X_SIZE = 0.2075665
# Y_SIZE = 0.2075665
# Z_SIZE = 0.4355092

In [ ]:
# Save face, edge, vert dataframe from initial segmentation
SAVE_INIT_SEGMENTATION = True
# Save face, edge, vert image from initial segmentation
SAVE_INIT_SEGMENTATION_IMAGE = True

SAVE_SIGNAL_MEASURE = True

DIRECT_VISUALISATION_SKELETON = True
DIRECT_VISUALISATION_VERTEX_EDGES = True
DIRECT_VISUALISATION_FACE = True

# Analyse morphologique
MORPHOLOGICAL_ANALYSIS = True
SAVE_MORPHOLOGICAL_ANALYSIS = True
AREA = True
PERIMETER = True
NB_NEIGHBOR = True
ANISO = True
J_ORIENTATION = True
DIRECT_VISUALISATION_ORIENTATION = True
DIRECT_VISUALISATION_SIGNAL = True

# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
THICKNESS = 0.5
DILATION = 3

SAVE_MORPHOLOGICAL_ANALYSIS = True
SAVE_INTENSITY_ANALYSIS = True

# Import 

In [ ]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits,
                         save_image,
                        )

from Dissects.image import (z_project,
                            thinning,
                            dilation)

from Dissects.geometry import Skeleton

from Dissects.draw.plt_draw import (plot_skeleton_3D,
                                    plot_face_3D,
                                    plot_junction_3D,
                                   )

from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )

from Dissects.analysis.analysis_3D_apical import (junction_intensity,
                                                  morphology_analysis)

from Dissects.segmentation.seg_3D_apical import (generate_segmentation)


In [ ]:
import os
import time

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
from skimage import morphology

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go
import pylab as pl

sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

In [ ]:
import matplotlib as mpl

# update fig parameter
mpl.rcParams['figure.dpi'] = 150 #300
mpl.rcParams['figure.figsize'] = (10,10)

In [ ]:
# get original image
img0 = load_image(os.path.join(directory, 
                              IMAGE_NAME))
img_myosin = io.imread(os.path.join(directory, 
                              IMAGE_NAME_MYOSIN))

In [ ]:
cp, fil, point, specs, _ = load_NDskl(os.path.join(directory, SKELETON_NAME))

# #create skeleton object
skel = Skeleton(cp, fil, point, specs)
skel_init = Skeleton(cp, fil, point, specs)

In [ ]:
#Clean skeleton
start = time.time()

skel.remove_lonely_cp()
skel.remove_free_filament()

end = time.time()
print(end-start)

In [ ]:
# Get new global variable
X_SHAPE = img0[0].shape[2]
Y_SHAPE = img0[0].shape[1]
Z_SHAPE = img0[0].shape[0]


In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    s = ndimage.generate_binary_structure(3,3)
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 
               IMAGE_NAME[:-4] + '_output_skeleton.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
if DIRECT_VISUALISATION_SKELETON:
    fig = plot_skeleton_3D(skel,
                          **{"x_size":X_SIZE,
                             "y_size":Y_SIZE,
                             "z_size":Z_SIZE,})
    fig.show()


# 3D vertex detection

In [ ]:
start = time.time()
face_df, edge_df, vert_df, points_df = generate_segmentation(skel, img0, **{"X_SIZE":X_SIZE,
                                                                           "Y_SIZE":Y_SIZE,
                                                                           "Z_SIZE":Z_SIZE})
end= time.time()
print(end-start)

if SAVE_INIT_SEGMENTATION : 
    vert_df.to_csv(os.path.join(directory, 'vert_df.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df.csv'))


### Save few outputs

In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    
    # Vertex image
    vertex_image = np.zeros([Z_SHAPE, Y_SHAPE, X_SHAPE])
    s = ndimage.generate_binary_structure(3,3)

    vertex_image[vert_df.z_pix.to_numpy().astype(int),
                 vert_df.y_pix.to_numpy().astype(int),
                 vert_df.x_pix.to_numpy().astype(int)] = 1
    # Dilate a little bit vertex to have a better viewing
    vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)
    
    save_image(vertex_image, 
               IMAGE_NAME[:-4] + '_output_vertex.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})
    
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 
               IMAGE_NAME[:-4] + '_output_skeleton.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
# Tiff cell identity 
saving_image = np.zeros(enlarge_face.shape)
for e in edge_df.index:        
    saving_image[list(edge_df.loc[e, 'point_z'])[0].astype(int),
                list(edge_df.loc[e, 'point_y'])[0].astype(int),
                list(edge_df.loc[e, 'point_x'])[0].astype(int)] = e+1


save_image(saving_image, 
           IMAGE_NAME[:-4] + '_output_edge.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})


In [ ]:
from Dissects.analysis.analysis_3D_apical import face_intensity

enlarge_face = face_intensity(img_myosin, 
                              face_df, 
                              edge_df, 
                              vert_df,
                              points_df, 
                              THICKNESS, 
                              DILATION, 
                              {"X_SIZE":X_SIZE,
                               "Y_SIZE":Y_SIZE,
                               "Z_SIZE":Z_SIZE})

save_image(enlarge_face, 
           IMAGE_NAME[:-4] + '_output_enlarge_face_identity.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})

In [ ]:
enlarge_face_random = np.zeros(enlarge_face.shape)
unique_value = np.unique(enlarge_face)
replace_value=[]
for i in range(len(unique_value)+1):
    rand_val = np.random.randint(10,2**8)
    while rand_val in replace_value:
        rand_val = np.random.randint(10,2**8)
    replace_value.append(rand_val)
    
for i in range(1, len(unique_value)):
    pos = np.where(enlarge_face==unique_value[i])
    enlarge_face_random[pos] = replace_value[i]

save_image(enlarge_face, 
           IMAGE_NAME[:-4] + '_output_enlarge_face_identity_random.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})

### Direct visualisation

In [ ]:
import random 
if DIRECT_VISUALISATION_FACE:
    fig = plot_skeleton_3D(skel,
                          **{"x_size":X_SIZE,
                             "y_size":Y_SIZE,
                             "z_size":Z_SIZE,})

    fig = plot_face_3D(skel,
                       face_df,
                       edge_df,
                       vert_df, 
                       fig,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})
    fig.show()

In [ ]:
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    fig = plot_skeleton_3D(skel,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})

 
    fig = plot_junction_3D(skel,
                       face_df,
                       edge_df,
                       vert_df, 
                       fig,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})
    fig.show()
    


# Analyse

In [ ]:
# morphometry analysis

if MORPHOLOGICAL_ANALYSIS :
    morphology_analysis(face_df, 
                        edge_df, 
                        vert_df, 
                        points_df,
                        area=AREA,
                        perimeter=PERIMETER,
                        nb_neighbor=NB_NEIGHBOR,
                        aniso=ANISO,
                        j_orientation=J_ORIENTATION,
                       )
face_df.drop(-1, axis=0, inplace=True)                        

In [ ]:
if SAVE_MORPHOLOGICAL_ANALYSIS : 
    vert_df.to_csv(os.path.join(directory, 'vert_df_morphology.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df_morphology.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df_morphology.csv'))

In [ ]:
if DIRECT_VISUALISATION_ORIENTATION:
    image_skeleton = skel.create_binary_image()
    z0, y0,x0 = np.where(image_skeleton>0)
    background_skeleton = go.Scatter3d(x=x0,
                                        y=y0,
                                        z=z0,
                                        mode='markers',
                                        marker=dict(
                                            size=2,
                                            color='black',
                                            opacity=0.1
                                         )
                        )
    
    
    
    fig = go.Figure(data=[background_skeleton], )
    fig.update_layout(title='Background-Aniso', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )
    
    # Edges
    cmap = pl.cm.cividis
    max_aniso = face_df['aniso'][1:].max()
    face_df['norm_aniso'] = (face_df['aniso']/max_aniso)
    factor = 2
    startx = (face_df['fx'] - factor*face_df['norm_aniso']*face_df['orientationx'])/X_SIZE
    starty = (face_df['fy'] - factor*face_df['norm_aniso']*face_df['orientationy'])/Y_SIZE
    startz = (face_df['fz'] - factor*face_df['norm_aniso']*face_df['orientationz'])/Z_SIZE
    endx = (face_df['fx'] + factor*face_df['norm_aniso']*face_df['orientationx'])/X_SIZE
    endy = (face_df['fy'] + factor*face_df['norm_aniso']*face_df['orientationy'])/Y_SIZE
    endz = (face_df['fz'] + factor*face_df['norm_aniso']*face_df['orientationz'])/Z_SIZE
    
    for i in face_df.index[1:]:
        fig.add_trace(
                go.Scatter3d(
                    x=[startx[i], endx[i]],
                    y=[starty[i], endy[i]],
                    z=[startz[i], endz[i]],
                    mode='lines',
                    line={#"color":'red',
                          "color":'rgb'+str(cmap(int(face_df.loc[i]['norm_aniso']*cmap.N))[0:3]),
                          "width":10,
                         },                
                )
            )
        
    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()
    

In [ ]:

from tifffile import tifffile

def save_image_analyse_face(enlarge_face,
                   face_df, 
                   column,
                   normalize=True,
                   normalize_max=None):
    
    if normalize:
        if normalize_max is None:
            face_df[column+'_norm'] = face_df[column]/face_df[column].max()
        else:
            face_df[column+'_norm'] = face_df[column]/normalize_max
        
    
    saving_image = np.zeros(enlarge_face.shape)
    
    for f in face_df.index:
        face_position = np.where(enlarge_face==f+1)
        if normalize:
            saving_image[face_position] = face_df.loc[f,column+'_norm']
        else:
            saving_image[face_position] = face_df.loc[f,column]
    
    save_image(saving_image, 
               IMAGE_NAME[:-4] +'_'+ column +'.tif',
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})


# Tiff cell identity 
save_image_analyse_face(enlarge_face, face_df, 'perimeter_approximate', normalize=False)

In [ ]:
def save_image_aniso(skeleton,
                    face_df,
                    edge_df,
                    vert_df,
                    normalize=True,
                    normalize_max=None,
                    factor=2):
    
    from skimage.draw import line_nd
    from scipy.ndimage import grey_dilation
    
    image_skeleton = skeleton.create_binary_image()
    image_skeleton = np.zeros(image_skeleton.shape)
    if normalize:
        if normalize_max is None:
            face_df['aniso_norm'] = face_df['aniso']/face_df['aniso'].max()
        else:
            face_df['aniso_norm'] = face_df['aniso']/normalize_max
            
    startx = ((face_df['fx'] - factor*face_df['aniso_norm']*face_df['orientationx'])/X_SIZE).to_numpy().astype(int)
    starty = ((face_df['fy'] - factor*face_df['aniso_norm']*face_df['orientationy'])/Y_SIZE).to_numpy().astype(int)
    startz = ((face_df['fz'] - factor*face_df['aniso_norm']*face_df['orientationz'])/Z_SIZE).to_numpy().astype(int)
    endx = ((face_df['fx'] + factor*face_df['aniso_norm']*face_df['orientationx'])/X_SIZE).to_numpy().astype(int)
    endy = ((face_df['fy'] + factor*face_df['aniso_norm']*face_df['orientationy'])/Y_SIZE).to_numpy().astype(int)
    endz = ((face_df['fz'] + factor*face_df['aniso_norm']*face_df['orientationz'])/Z_SIZE).to_numpy().astype(int)
    
    for i in face_df.index:
        
        tmp_image = np.zeros(image_skeleton.shape)
        coords = line_nd((startx[i-1], starty[i-1], startz[i-1]),
                         (endx[i-1], endy[i-1], endz[i-1]), 
                         endpoint=True,
                         integer=True)
        
        tmp_image[coords[2],
               coords[1], 
               coords[0]] = 1
        #enlarge
        s = sci.ndimage.generate_binary_structure(3, 3)
        
        tmp_image = sci.ndimage.morphology.binary_dilation(
                                     tmp_image, s).astype(int)
        
        pos = np.where(tmp_image==1)
        image_skeleton[pos] = face_df.loc[i, 'aniso_norm']
 

    save_image(image_skeleton, 
               IMAGE_NAME[:-4] +'_aniso_orientation.tif',
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
save_image_aniso(skel, 
                face_df, 
                edge_df,
                vert_df, 
                factor=4)